In [ ]:
#!pip install mysql-connector-python

import os
import pandas as pd
import mysql.connector
from  mysql.connector import errorcode

In [ ]:
def insertar_datos_peliculas():

    """
    Esta función convierte csv con datos de peliculas a una lisat de tuplas y las inserta en una bbdd
    
    Args:
    - no tiene parametros

    Returns:
    - devuelve el número de registros insertados
    """

    lista_tuplas_pelis = []        
    for i in range(1990,2024):
        for j in range(1,4):
            try:
                df_pelis = pd.read_csv(f'datos/datos_peliculas/lista_peliculas_{i}_{j}000.csv')
                
                orden_pelis = ["0","1","2","3","4","5","6","9","10"]
            
                df_peliculas = df_pelis.reindex(columns = orden_pelis)
                
                #quitar nan y null del dataframe
                df_final_peliculas = df_peliculas.dropna(how='any', axis=0)
                               
                for fila in df_final_peliculas.values:
                    lista_tuplas_pelis.append(tuple(fila))
            except:
                print(f"lista_peliculas_{i}_{j}000.csv - no existe")            
                break
           
    #Tabla peliculas

    try:
  
        # Recuperamos la contraseña del .env
        contrasena = os.environ.get('contraseña')
  
        #concexion
        cnx = mysql.connector.connect(user="root",
                        password=contrasena,
                        host="localhost", database = "bho_peliculas_3")
        
    except mysql.connector.Error as err:

        # si es un error con la contraseña devuelveme un mensaje de acceso denegado ya que tenemos problemas con la contraseña
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        
        # si el error no tiene que ver con la contraseña, puede ser porque la base de datos no exista, devuelveme un mensaje de que la base de datos no existe
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        
        # si no es por ninguno de los errores anteriores, printeame cual es el error que estoy teniendo en mi conexión
        else:
            print(err)

    

    mycursor = cnx.cursor()

    sql_peliculas = "INSERT INTO peliculas (id_pelicula, nombre_pelicula, tipo, genero, año_estreno, mes_estreno, direccion, argumento, duracion) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"

    try:
        mycursor.executemany(sql_peliculas, set(lista_tuplas_pelis))
        cnx.commit()
        print(mycursor.rowcount, "registros insertados")
    
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
        print("peliculas - error inserción")

    cnx.close()    
    
    return mycursor.rowcount        

In [ ]:
insertar_datos_peliculas()